# Are there any routes on which rail leads to shorter journey times than air travel?

## Data Exploration

In [34]:
import pandas as pd
import folium
import os
from folium import FeatureGroup, LayerControl

In [2]:
DATA_DIR = "../Data"

In [3]:
airport_indicators = pd.read_excel(os.path.join(DATA_DIR, "Airports_Indicators.xlsx"))
airport_rail_access = pd.read_excel(os.path.join(DATA_DIR, "Airports_Rail_Access.xlsx"))
alt_rail_routes_bidirectional = pd.read_excel(os.path.join(DATA_DIR, "Alternative_Rail_Routes_Bidirectional.xlsx"))
alt_rail_routes_unidirectional = pd.read_excel(os.path.join(DATA_DIR, "Alternative_Rail_Routes_Unidirectional.xlsx"))
candidate_rail_routes = pd.read_excel(os.path.join(DATA_DIR, "Candidate_Rail_Routes.xlsx"))

In [4]:
airport_indicators.head()

,Airport,IATA Code,Latitude,Longitude,Potential Passengers,Total Passengers,Connecting Passengers,Service ARISP,Theoretical ARISP,Delta ARISP
0,Aalborg Airport,AAL,57.086435,9.867982,58777,1460146,475826,6.751829e+03,1.373080e+04,6.978976e+03
1,Málaga-Costa del Sol Airport,AGP,36.678545,-4.499955,658614,20401883,2435885,1.160782e+05,2.157523e+05,9.967401e+04
2,Amsterdam Airport Schiphol,AMS,52.326980,4.741505,11181707,48283340,17605076,2.916196e+06,4.722534e+06,1.806338e+06
3,Marche Airport,AOI,43.617162,13.364858,8593,646447,81104,0.000000e+00,3.102980e+03,3.102980e+03
4,Stockholm-Arlanda Airport,ARN,59.646792,17.937044,2809610,20200389,5585200,5.152913e+05,8.919868e+05,3.766954e+05


In [5]:
airport_rail_access.columns = airport_rail_access.iloc[0]
airport_rail_access = airport_rail_access[1:]
airport_rail_access.head()

,Airport,IATA Code,Ptp Passengers,Spoke Passengers,Hub Passengers,Ptp Passengers [%],Spoke Passengers [%],Hub Passengers [%],Total Passengers,Generated Passengers,...,Detour Line,Branch Line,Linked to Line,High-Speed Rail,Long-distance,Regional/suburban,City-Airport Express Train,Train connection to City Station,Metro/Tram/ People Mover,Comments
1,Frankfurt Airport,FRA,21173848,6305886,15964490,0.48738,0.145149,0.367471,43444224,27479734,...,0,0,0,1,1,1,0,"ICE (12 min, nonstop)",0,NaN
2,Charles de Gaulle International Airport,CDG,32720275,9206483,11706349,0.610076,0.171657,0.218267,53633107,41926758,...,0,0,0,1,1,1,0,RER-B (31 min),0,NaN
3,Amsterdam Airport Schiphol,AMS,30678264,6169779,11435297,0.63538,0.127783,0.236837,48283340,36848043,...,0,0,0,1,1,1,0,"Intercity Direct (15 min, nonstop)",0,NaN
4,London Heathrow Airport,LHR,41329628,13195766,11295771,0.627908,0.200479,0.171613,65821165,54525394,...,0,1,0,0,0,1,1,"Heathrow Express (21 min, nonstop)",1,NaN
5,Madrid–Barajas Airport,MAD,32762382,5861990,9799351,0.676577,0.121056,0.202367,48423723,38624372,...,0,1,0,0,0,1,0,"Carcanias C1, C10 (19 min)",1,NaN


In [6]:
airport_rail_access.describe()

,Airport,IATA Code,Ptp Passengers,Spoke Passengers,Hub Passengers,Ptp Passengers [%],Spoke Passengers [%],Hub Passengers [%],Total Passengers,Generated Passengers,...,Detour Line,Branch Line,Linked to Line,High-Speed Rail,Long-distance,Regional/suburban,City-Airport Express Train,Train connection to City Station,Metro/Tram/ People Mover,Comments
count,318,318,318,318,318,318,318,318,318,318,...,318,318,318,318,318,318,318,318,318,9
unique,317,317,317,307,236,308,308,249,317,317,...,3,3,3,3,3,2,3,64,2,9
top,Düsseldorf Airport,DUS,14630079,0,0,1,0,0,18536792,18003273,...,-,-,-,-,-,-,-,-,0,MetroLink station planned to open in 2035
freq,2,2,2,10,69,10,10,69,2,2,...,234,234,234,234,234,234,255,255,281,1


In [7]:
alt_rail_routes_bidirectional.head()

,Bidirectional Route Name,Origin,Destination,Origin Coordinates,Destination Coordinates,Rail Theoretical Travel Time,Rail Service Travel Time,Air Travel Time,Valued Air Travel Time,Passenger Flows,Theoretical ARISP,Service ARISP,Theoretical Time Difference,Service Time Difference,Theoretical Time Increase,Service Time Increase
0,A Coruña-AGP,A Coruña,AGP,"43.352681,-8.409651","36.677749,-4.490162",466.960144,471.064213,100.010468,162.016959,3613,1253.570094,1242.648572,-304.943185,-309.047254,188.216831,190.749942
1,A Coruña-BCN,A Coruña,BCN,"43.352681,-8.409651","41.304141,2.073126",456.163626,867.916176,104.935906,169.996167,34310,12786.132358,6720.197937,-286.167459,-697.920009,168.337595,410.550438
2,A Coruña-GVA,A Coruña,GVA,"43.352681,-8.409651","46.231334,6.110271",813.577093,964.708150,135.000000,218.700000,908,244.082094,205.844224,-594.877093,-746.008150,272.005987,341.110265
3,A Coruña-LGW,A Coruña,LGW,"43.352681,-8.409651","51.156735,-0.160922",885.255404,1250.218575,119.621596,193.786986,4996,1093.650236,774.392415,-691.468418,-1056.431589,356.818811,545.150947
4,A Coruña-MAD,A Coruña,MAD,"43.352681,-8.409651","40.492435,-3.592839",317.769684,273.757594,76.021220,123.154376,184030,71322.410288,82788.935567,-194.615308,-150.603218,158.025492,122.288158


In [8]:
alt_rail_routes_unidirectional.head()

,Route Name,Origin,Destination,Origin Coordinates,Destination Coordinates,Rail Theoretical Travel Time,Rail Service Travel Time,Air Travel Time,Valued Air Travel Time,Valued Air Travel Time_MIN,...,Theoretical Time Difference_MIN,Service Time Difference_MIN,Theoretical Time Difference_MAX,Service Time Difference_MAX,Theoretical Time Increase,Service Time Increase,Theoretical Time Increase_MIN,Service Time Increase_MIN,Theoretical Time Increase_MAX,Service Time Increase_MAX
0,A Coruña-AGP,A Coruña,Málaga-Costa del Sol Airport,"43.352681,-8.409651","36.677749,-4.490162",467.0,471.0,100.000000,162.000000,141.000000,...,-326.000000,-330.000000,-284.00000,-288.00000,188.271605,190.740741,231.205674,234.042553,155.191257,157.377049
1,A Coruña-BCN,A Coruña,Barcelona-El Prat Airport,"43.352681,-8.409651","41.304141,2.073126",473.0,845.0,100.000000,162.000000,141.000000,...,-332.000000,-704.000000,-290.00000,-662.00000,191.975309,421.604938,235.460993,499.290780,158.469945,361.748634
2,A Coruña-GVA,A Coruña,Geneva Cointrin International Airport,"43.352681,-8.409651","46.231334,6.110271",824.0,961.0,135.000000,218.700000,190.350000,...,-633.650000,-770.650000,-576.95000,-713.95000,276.771834,339.414723,332.886787,404.859469,233.535722,288.990083
3,A Coruña-LGW,A Coruña,London Gatwick Airport,"43.352681,-8.409651","51.156735,-0.160922",910.0,1241.0,117.979798,191.127273,166.351515,...,-743.648485,-1074.648485,-694.09697,-1025.09697,376.122527,549.305556,447.034392,646.010638,321.485515,474.795082
4,A Coruña-MAD,A Coruña,Madrid–Barajas Airport,"43.352681,-8.409651","40.492435,-3.592839",340.0,242.0,75.131711,121.713371,105.935712,...,-234.064288,-136.064288,-202.50897,-104.50897,179.344822,98.827785,220.949370,128.440434,147.288859,76.011482


In [9]:
candidate_rail_routes.head()

,Airport OD,Origin,Destination,Origin Coordinates,Destination Coordinates,Route,Distance and Theoretical Time,Theoretical Travel Time,Distance,Service Travel Time,Mode
0,LHR-FAO,London Heathrow Airport,Faro,"51.472222,-0.487725","37.019187,-7.940261","Linha do Minho, Linha do Norte - Via Descendente","2973.7 km, 20 h 31 min",20:31:00,2973.7 km,-,-
1,FAO-LHR,Faro,London Heathrow Airport,"37.019187,-7.940261","51.472222,-0.487725","Linha do Sul, Madril-Hendaia trenbidea/Línea M...","2723.4 km, 23 h 32 min",23:32:00,2723.4 km,-,-
2,LHR-FLR,London Heathrow Airport,Firenze,"51.472222,-0.487725","43.77681,11.247806","Ligne de Combs-la-Ville à Saint-Louis (LGV), L...","1691.1 km, 8 h 19 min",08:19:00,1691.1 km,14h 56min,Train
3,FLR-LHR,Firenze,London Heathrow Airport,"43.77681,11.247806","51.472222,-0.487725","Ferrovia Milano-Bologna AV/AC, Ligne de Paris ...","1724.0 km, 8 h 31 min",08:31:00,1724.0 km,14h 45min,Train
4,LHR-LDY,London Heathrow Airport,Derry,"51.472222,-0.487725",NaN,-,-,-,-,-,-


In [10]:
# Filter for Rail Service faster than Valued Air
faster_rail_service_valued = alt_rail_routes_unidirectional[
    alt_rail_routes_unidirectional['Service Time Difference'] > 0]
num_faster_rail_service_valued = len(faster_rail_service_valued)
num_faster_rail_service_valued


1

In [11]:
# Filter for Rail Theoretical faster than Valued Air
faster_rail_theoretical_valued = alt_rail_routes_unidirectional[
    alt_rail_routes_unidirectional['Theoretical Time Difference'] > 0]
num_faster_rail_theoretical_valued = len(faster_rail_theoretical_valued)
num_faster_rail_theoretical_valued

35

In [12]:
alt_rail_routes_unidirectional['Service Time Difference Raw'] = (
        alt_rail_routes_unidirectional['Air Travel Time'] - alt_rail_routes_unidirectional['Rail Service Travel Time'])
faster_rail_service_raw = (
    alt_rail_routes_unidirectional)[alt_rail_routes_unidirectional['Service Time Difference Raw'] > 0]
num_faster_rail_service_raw = len(faster_rail_service_raw)
num_faster_rail_service_raw

0

In [13]:
alt_rail_routes_unidirectional['theoretical_vs_raw'] = alt_rail_routes_unidirectional['Air Travel Time'] - \
                                                       alt_rail_routes_unidirectional['Rail Theoretical Travel Time']
theoretical_vs_raw = alt_rail_routes_unidirectional[
    alt_rail_routes_unidirectional['theoretical_vs_raw'] > 0]
num_faster_theoretical_raw = len(theoretical_vs_raw)
num_faster_theoretical_raw

0

In [14]:
examples = faster_rail_service_valued[
    ['Route Name', 'Origin', 'Destination', 'Rail Service Travel Time',
     'Valued Air Travel Time', 'Service Time Difference', 'Air Travel Time']].head(10)

print(f"Number of routes where Rail Service is faster than Valued Air: {num_faster_rail_service_valued}")
print(f"Number of routes where Rail Theoretical is faster than Valued Air: {num_faster_rail_theoretical_valued}")
print(f"Number of routes where Rail Service is faster than Raw Air Time: {num_faster_rail_service_raw}")
print(f"Number of routes where Rail Theoretical Travel Time is faster than Raw Air Time: {num_faster_theoretical_raw}")
print("\nExamples where Rail Service is faster than Valued Air:")
examples

Number of routes where Rail Service is faster than Valued Air: 1
Number of routes where Rail Theoretical is faster than Valued Air: 35
Number of routes where Rail Service is faster than Raw Air Time: 0
Number of routes where Rail Theoretical Travel Time is faster than Raw Air Time: 0

Examples where Rail Service is faster than Valued Air:


,Route Name,Origin,Destination,Rail Service Travel Time,Valued Air Travel Time,Service Time Difference,Air Travel Time
6373,Stuttgart-FRA,Stuttgart,Frankfurt Airport,71.0,81.0,10.0,50.0


In [15]:
candidate_rail_routes.loc[
    (candidate_rail_routes['Origin'] == 'Stuttgart') & (candidate_rail_routes['Destination'] == 'Frankfurt Airport')]

,Airport OD,Origin,Destination,Origin Coordinates,Destination Coordinates,Route,Distance and Theoretical Time,Theoretical Travel Time,Distance,Service Travel Time,Mode
2117,STR-FRA,Stuttgart,Frankfurt Airport,"48.784081,9.181636","50.052932,8.569915","Schnellfahrstrecke Mannheim–Stuttgart, Main-Ne...","182.1 km, 1 h 11 min",01:11:00,182.1 km,1h 11min,Train


In [16]:
faster_rail_theoretical_valued[['Route Name', 'Origin', 'Destination', 'Rail Theoretical Travel Time',
                                'Valued Air Travel Time', 'Theoretical Time Difference']]

,Route Name,Origin,Destination,Rail Theoretical Travel Time,Valued Air Travel Time,Theoretical Time Difference
181,AMS-BRU,Amsterdam Airport Schiphol,Brussels Airport,66.0,73.576865,7.576865
194,AMS-Brussels,Amsterdam Airport Schiphol,Brussels,73.0,73.576865,0.576865
1171,BRU-AMS,Brussels Airport,Amsterdam Airport Schiphol,66.0,90.668465,24.668465
1176,BRU-Amsterdam,Brussels Airport,Amsterdam,77.0,90.668465,13.668465
1193,BRU-CDG,Brussels Airport,Charles de Gaulle International Airport,78.0,101.690722,23.690722
1221,BRU-London,Brussels Airport,London,111.0,119.661796,8.661796
1238,BRU-Paris,Brussels Airport,Paris,87.0,101.690722,14.690722
1676,Brussels-AMS,Brussels,Amsterdam Airport Schiphol,74.0,90.668465,16.668465
1683,Brussels-CDG,Brussels,Charles de Gaulle International Airport,64.0,101.690722,37.690722
1806,CDG-BRU,Charles de Gaulle International Airport,Brussels Airport,75.0,96.382569,21.382569


In [22]:

faster_rail_theoretical_valued = faster_rail_theoretical_valued.copy()

def split_coords(coord_str):
    lat_str, lon_str = coord_str.split(',')
    return float(lat_str.strip()), float(lon_str.strip())

faster_rail_theoretical_valued['origin_lat'] = faster_rail_theoretical_valued['Origin Coordinates'].map(lambda x: split_coords(x)[0])
faster_rail_theoretical_valued['origin_lon'] = faster_rail_theoretical_valued['Origin Coordinates'].map(lambda x: split_coords(x)[1])
faster_rail_theoretical_valued['dest_lat'] = faster_rail_theoretical_valued['Destination Coordinates'].map(lambda x: split_coords(x)[0])
faster_rail_theoretical_valued['dest_lon'] = faster_rail_theoretical_valued['Destination Coordinates'].map(lambda x: split_coords(x)[1])

In [23]:
faster_rail_theoretical_valued

,Route Name,Origin,Destination,Origin Coordinates,Destination Coordinates,Rail Theoretical Travel Time,Rail Service Travel Time,Air Travel Time,Valued Air Travel Time,Valued Air Travel Time_MIN,...,Theoretical Time Increase,Service Time Increase,Theoretical Time Increase_MIN,Service Time Increase_MIN,Theoretical Time Increase_MAX,Service Time Increase_MAX,origin_lat,origin_lon,dest_lat,dest_lon
181,AMS-BRU,Amsterdam Airport Schiphol,Brussels Airport,"52.309523,4.762181","50.898344,4.482370",66.0,131.0,45.417818,73.576865,64.039123,...,-10.297890,78.045097,3.061999,104.562452,-20.591575,57.613692,52.309523,4.762181,50.898344,4.482370
194,AMS-Brussels,Amsterdam Airport Schiphol,Brussels,"52.309523,4.762181","50.835374,4.335695",73.0,99.0,45.417818,73.576865,64.039123,...,-0.784030,34.553165,13.992817,54.592998,-12.169469,19.112638,52.309523,4.762181,50.835374,4.335695
1171,BRU-AMS,Brussels Airport,Amsterdam Airport Schiphol,"50.898344,4.482370","52.309523,4.762181",66.0,145.0,55.968188,90.668465,78.915145,...,-27.207326,59.923299,-16.365864,83.741663,-35.560584,41.571445,50.898344,4.482370,52.309523,4.762181
1176,BRU-Amsterdam,Brussels Airport,Amsterdam,"50.898344,4.482370","52.37919,4.899426",77.0,163.0,55.968188,90.668465,78.915145,...,-15.075214,79.775847,-2.426841,106.550973,-24.820681,59.145831,50.898344,4.482370,52.379190,4.899426
1193,BRU-CDG,Brussels Airport,Charles de Gaulle International Airport,"50.898344,4.482370","49.003989,2.571154",78.0,190.0,62.772050,101.690722,88.508591,...,-23.296837,86.841038,-11.872962,114.668427,-32.098839,65.400263,50.898344,4.482370,49.003989,2.571154
1221,BRU-London,Brussels Airport,London,"50.898344,4.482370","51.531921,-0.126361",111.0,204.0,73.865306,119.661796,104.150082,...,-7.238564,70.480477,6.576969,95.871186,-17.883319,50.917143,50.898344,4.482370,51.531921,-0.126361
1238,BRU-Paris,Brussels Airport,Paris,"50.898344,4.482370","48.8804,2.354973",87.0,125.0,62.772050,101.690722,88.508591,...,-14.446472,22.921736,-1.704457,41.229228,-24.264090,8.815963,50.898344,4.482370,48.880400,2.354973
1676,Brussels-AMS,Brussels,Amsterdam Airport Schiphol,"50.835374,4.335695","52.309523,4.762181",74.0,97.0,55.968188,90.668465,78.915145,...,-18.383972,6.983172,-6.228393,22.916836,-27.749745,-5.293585,50.835374,4.335695,52.309523,4.762181
1683,Brussels-CDG,Brussels,Charles de Gaulle International Airport,"50.835374,4.335695","49.003989,2.571154",64.0,107.0,62.772050,101.690722,88.508591,...,-37.064071,5.221006,-27.690635,20.892219,-44.286227,-6.853536,50.835374,4.335695,49.003989,2.571154
1806,CDG-BRU,Charles de Gaulle International Airport,Brussels Airport,"49.003989,2.571154","50.898344,4.482370",75.0,197.0,59.495413,96.382569,83.888532,...,-22.185100,104.393805,-10.595646,134.835436,-31.114678,80.938778,49.003989,2.571154,50.898344,4.482370


In [25]:
full_info_faster_rail_theoretical_valued = faster_rail_theoretical_valued.merge(
    candidate_rail_routes,
    left_on=['Origin','Destination'],
    right_on=['Origin','Destination'],
    how='left'
)

In [30]:
full_info_faster_rail_theoretical_valued['Distance'] = full_info_faster_rail_theoretical_valued['Distance'].str[:-3].astype(float)

In [31]:
full_info_faster_rail_theoretical_valued

,Route Name,Origin,Destination,Origin Coordinates_x,Destination Coordinates_x,Rail Theoretical Travel Time,Rail Service Travel Time,Air Travel Time,Valued Air Travel Time,Valued Air Travel Time_MIN,...,dest_lon,Airport OD,Origin Coordinates_y,Destination Coordinates_y,Route,Distance and Theoretical Time,Theoretical Travel Time,Distance,Service Travel Time,Mode
0,AMS-BRU,Amsterdam Airport Schiphol,Brussels Airport,"52.309523,4.762181","50.898344,4.482370",66.0,131.0,45.417818,73.576865,64.039123,...,4.482370,AMS-BRU,"52.309523,4.762181","50.898344,4.482370","HSL-Zuid, L25","185.8 km, 1 h 6 min",01:06:00,185.8,2h 11min,Train
1,AMS-Brussels,Amsterdam Airport Schiphol,Brussels,"52.309523,4.762181","50.835374,4.335695",73.0,99.0,45.417818,73.576865,64.039123,...,4.335695,AMS-BRU,"52.309523,4.762181","50.835374,4.335695","HSL-Zuid, L25","195.1 km, 1 h 13 min",01:13:00,195.1,1h 39min,Train
2,BRU-AMS,Brussels Airport,Amsterdam Airport Schiphol,"50.898344,4.482370","52.309523,4.762181",66.0,145.0,55.968188,90.668465,78.915145,...,4.762181,BRU-AMS,"50.898344,4.482370","52.309523,4.762181","L25, HSL-Zuid","185.8 km, 1 h 6 min",01:06:00,185.8,2h 25min,Train
3,BRU-Amsterdam,Brussels Airport,Amsterdam,"50.898344,4.482370","52.37919,4.899426",77.0,163.0,55.968188,90.668465,78.915145,...,4.899426,BRU-AMS,"50.898344,4.482370","52.37919,4.899426","L25, HSL-Zuid","202.7 km, 1 h 17 min",01:17:00,202.7,2h 43min,Train
4,BRU-CDG,Brussels Airport,Charles de Gaulle International Airport,"50.898344,4.482370","49.003989,2.571154",78.0,190.0,62.772050,101.690722,88.508591,...,2.571154,BRU-CDG,"50.898344,4.482370","49.003989,2.571154","L1, Ligne de Gonesse à Lille-Frontière (LGV)","306.5 km, 1 h 18 min",01:18:00,306.5,3h 10min,Train
5,BRU-London,Brussels Airport,London,"50.898344,4.482370","51.531921,-0.126361",111.0,204.0,73.865306,119.661796,104.150082,...,-0.126361,BRU-LHR,"50.898344,4.482370","51.531921,-0.126361","Ligne de Fretin à Frethun (LGV), High Speed 1","387.7 km, 1 h 51 min",01:51:00,387.7,3h 24min,Train
6,BRU-Paris,Brussels Airport,Paris,"50.898344,4.482370","48.8804,2.354973",87.0,125.0,62.772050,101.690722,88.508591,...,2.354973,BRU-CDG,"50.898344,4.482370","48.8804,2.354973","L1, Ligne de Gonesse à Lille-Frontière (LGV)","328.6 km, 1 h 27 min",01:27:00,328.6,2h 5min,Train
7,Brussels-AMS,Brussels,Amsterdam Airport Schiphol,"50.835374,4.335695","52.309523,4.762181",74.0,97.0,55.968188,90.668465,78.915145,...,4.762181,BRU-AMS,"50.835374,4.335695","52.309523,4.762181","L25, HSL-Zuid","196.7 km, 1 h 14 min",01:14:00,196.7,1h 37min,Train
8,Brussels-CDG,Brussels,Charles de Gaulle International Airport,"50.835374,4.335695","49.003989,2.571154",64.0,107.0,62.772050,101.690722,88.508591,...,2.571154,BRU-CDG,"50.835374,4.335695","49.003989,2.571154","L1, Ligne de Gonesse à Lille-Frontière (LGV)","291.0 km, 1 h 4 min",01:04:00,291.0,1h 47min,Train
9,CDG-BRU,Charles de Gaulle International Airport,Brussels Airport,"49.003989,2.571154","50.898344,4.482370",75.0,197.0,59.495413,96.382569,83.888532,...,4.482370,CDG-BRU,"49.003989,2.571154","50.898344,4.482370","Ligne de Gonesse à Lille-Frontière (LGV), L1","307.0 km, 1 h 15 min",01:15:00,307.0,3h 17min,Train


In [37]:
all_origins = set(df['Origin'])
all_destinations = set(df['Destination'])
hub_stations = all_origins.intersection(all_destinations)

m = folium.Map(location=[50.8503, 4.3517], zoom_start=6, tiles='CartoDB positron')

routes_layer = FeatureGroup(name="Connections")
stations_layer = FeatureGroup(name="Stations")

plotted_cities = set()

tooltip_style = "font-size: 14px; font-weight: bold; white-space: nowrap;"

for idx, row in df.iterrows():
    p1 = (row['origin_lat'], row['origin_lon'])
    p2 = (row['dest_lat'], row['dest_lon'])

    info_text = (f"{row['Origin']} to {row['Destination']}: "
                 f"Rail {row['Rail Theoretical Travel Time']} min, "
                 f"Air {round(row['Valued Air Travel Time'], 2)} min, "
                 f"Distance {row['Distance']} km")

    folium.PolyLine(
        locations=[p1, p2],
        color='green',
        weight=2,
        opacity=0.5,
        tooltip=folium.Tooltip(info_text, style=tooltip_style)
    ).add_to(routes_layer)


    if row['Origin'] not in plotted_cities:
        if row['Origin'] in hub_stations:
            icon_obj = folium.Icon(color='purple', icon='exchange', prefix='fa')
            popup_label = f"{row['Origin']} (Hub)"
        else:
            icon_obj = folium.Icon(color='green', icon='train', prefix='fa')
            popup_label = f"{row['Origin']} (Origin)"

        folium.Marker(
            location=p1,
            tooltip=folium.Tooltip(popup_label, style=tooltip_style), # Larger text
            icon=icon_obj
        ).add_to(stations_layer)
        plotted_cities.add(row['Origin'])

    if row['Destination'] not in plotted_cities:
        if row['Destination'] in hub_stations:

            icon_obj = folium.Icon(color='purple', icon='exchange', prefix='fa')
            popup_label = f"{row['Destination']} (Hub)"
        else:
            icon_obj = folium.Icon(color='red', icon='home', prefix='fa')
            popup_label = f"{row['Destination']} (Destination)"

        folium.Marker(
            location=p2,
            tooltip=folium.Tooltip(popup_label, style=tooltip_style), # Larger text
            icon=icon_obj
        ).add_to(stations_layer)
        plotted_cities.add(row['Destination'])

routes_layer.add_to(m)
stations_layer.add_to(m)

LayerControl().add_to(m)

m.save("map_osm.html")
print("Map saved as 'map_osm.html'. Open this file in your browser.")
m

Map saved as 'map_osm.html'. Open this file in your browser.
